# AIM: Extract features on labeled data using the pretrained EEGNet
across subject pretext task

In [1]:
import numpy as np
import pandas as pd
import mne
import lightning.pytorch as pl
import torch
import torch.nn.functional as F
import torch.nn as nn
from torchmetrics import F1Score, Accuracy
import random
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedGroupKFold
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from IPython.display import display
%matplotlib inline


# prevent extensive logging
mne.set_log_level('WARNING')

## Loading epoch data & participant data of labeled sample
These dataframes have been filtered and stored in a previous project. See https://github.com/TSmolders/Internship_EEG for original code

In [2]:

df_participants = pd.read_pickle(r'D:\Documents\RU\Master_Neurobiology\Internship_jaar_2\Project\TD-BRAIN\TDBRAIN_participants_V2_data\df_participants.pkl')
sample_df = pd.read_pickle(r'D:\Documents\RU\Master_Neurobiology\Internship_jaar_2\Project\TD-BRAIN\TD-BRAIN_extracted_features\df_selected_stat_features.pkl')
sample_ids = sample_df['ID'].unique() # obtain unique IDs from subsampled dataframe containing epoched features
df_sample = df_participants[df_participants['participants_ID'].isin(sample_ids)] # filter participants dataframe to only include subsampled IDs
df_sample = df_sample[df_sample['sessID'] == 1] # filter first session
print(df_sample.shape)
print(df_sample['diagnosis'].value_counts())



(225, 12)
diagnosis
ADHD       45
HEALTHY    45
MDD        45
OCD        45
SMC        45
Name: count, dtype: int64


In [3]:
# functions for loading the epoched EEG data
def get_filepath(epoch_dir, participant_ids):
    """
    Function to get the filepath of the epoched EEG recording
    :param epoch_dir: directory containing the epoched EEG recordings
    :param ID: list of participant IDs to include
    """
    filepaths = []
    for subdir, dirs, files in os.walk(epoch_dir):
        for file in files:
            if any(participant_id in file for participant_id in participant_ids):
                filepaths.append(os.path.join(subdir, file))
    return filepaths

class EpochDataset(torch.utils.data.Dataset):
    def __init__(self, participant_ids, epoch_dir):
        self.filepaths = get_filepath(epoch_dir, participant_ids)
        self.participant_ids = participant_ids
        self.epochs = []
        self.participant_ids = []
        self._load_data()
        print(f"Number of epochs: {self.epochs.shape[0]}")
        print(f"Number of participants: {len(self.participant_ids)}")

    def _load_data(self):
        all_epochs = []
        for filepath in self.filepaths:
            epochs = torch.load(filepath)
            # get participant ID from filepath to make sure the participant ID is correct
            participant_id = filepath.split("\\")[-1].split(".")[0]
            all_epochs.append(epochs)
            self.participant_ids.extend([participant_id]*epochs.shape[0])
        self.epochs = np.concatenate(all_epochs, axis=0)

    def __len__(self):
        return self.epochs.shape[0]
    
    def __getitem__(self, idx):
        epoch = self.epochs[idx]
        participant_id = self.participant_ids[idx]
        return torch.tensor(epoch, dtype=torch.float32), participant_id

In [4]:
# load the epochs into a dataset
participant_ids = df_sample['participants_ID'].tolist()
dataset = EpochDataset(participant_ids, r"D:\Documents\RU\Master_Neurobiology\Internship_jaar_2\Project\TD-BRAIN\TDBRAIN-dataset-derivatives\thesis_epoched_data\EC")
print(len(dataset))
print(dataset[0][0].shape)
print(dataset[0][1])
print(dataset[1][1])

Number of epochs: 2688
Number of participants: 2688
2688
torch.Size([26, 1244])
sub-87964717
sub-87964717


## Transfering pretrained weights & extracting features

### Functions:

In [5]:
def transfer_weights(pretrained_weights, pretext_model):
    """
    Function to transfer the pretrained weights to the pretext model
    param: pretrained_weights: the weights to transfer in a dictionary
    param: pretext_model: the model to transfer the weights to
    """
    pretrained_model = pretext_model
    modified_keys = {}
    for k, v in pretrained_weights.items():
        decomposed_key = k.split('.')
        if decomposed_key[0] == 'EEGNet' or decomposed_key[0] == 'ShallowNet': # remove the first part of each key to match the model's keys
            pretrained_key = '.'.join(decomposed_key[1:])
            modified_keys[pretrained_key] = v

            
    pretrained_model.load_state_dict(modified_keys)
        
    return pretrained_model

def extract_features(pretrained_model, data, pretext_task, df_sample, to_disk=False):
    """
    Function to extract features from the pretrained model
    param: pretrained_model: the model to extract features from
    param: data: the dataset containing the epochs to extract features from
    param: pretext_task: a string indicating the specific pretext task to save the features as
    param: df_sample: the dataframe containing the sampled participant IDs and their corresponding diagnosis
    param: to_disk: boolean to save the features to disk
    """
    dataloader = torch.utils.data.DataLoader(data, batch_size=1, shuffle=False)
    pretrained_model.eval()
    features_list = []
    participant_ids = []
    with torch.no_grad():  # Disable gradient calculation
        for batch in dataloader:
            epoch, participant_id = batch  # Remove the batch dimension
            epoch = epoch.unsqueeze(0)  # Add dimension
            # print(epoch.shape)
            features = pretrained_model(epoch)  # Extract features
            features = features.squeeze(0)
            features = features.numpy()
            features_list.append(features)
            participant_ids.append(participant_id[0])

    
    features_df = pd.DataFrame(features_list) # store as dataframe
    features_df['ID'] = participant_ids # add participant IDs to the dataframe
    # map the diagnosis values from df_sample to the dataframe based on participant IDs
    features_df['diagnosis'] = features_df['ID'].map(df_sample.set_index('participants_ID')['diagnosis'])
    
    print(f'{features_df.shape = }')
    display(features_df.head(3))


    if to_disk:
        features_df.to_pickle(f'D:/Documents/Master_Data_Science/Thesis/thesis_code/DataScience_Thesis/data/SSL_features/df_{pretext_task}_features.pkl')
    
    return features_df

def evaluate_features(features_df):
    """
    Function to quickly evaluate the extracted features. Doesn't stratify/group data splitting! 
    """
    groups = features_df['ID']
    X = features_df.drop(['ID', 'diagnosis'], axis=1)
    y = features_df['diagnosis']

    sgkf = StratifiedGroupKFold(n_splits=5, shuffle=False)

    # Get the train and test indices
    for train_index, test_index in sgkf.split(X, y, groups):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        break  # Only use the first split

    # quick SVM model
    clf = SVC()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('quick SVM model')
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(f1_score(y_test, y_pred, average='macro'))
    print()

    # quick random forest model
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('quick random forest model')
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(f1_score(y_test, y_pred, average='macro'))

    return

def get_ssl_features(
        pretext_task,
        data,
        df_sample,
        num_extracted_features=100,
        eval=True,
        to_disk=True,
        pretext_model='EEGNet',
        weights_dir=r'D:\Documents\Master_Data_Science\Thesis\thesis_code\DataScience_Thesis\data\pretext_model_weights'
                     ):
    """
    Obtains SSL features from the weights trained by the pretext model.
    param: pretext_task: a string indicating the specific pretext task to load the weights from and save the features as
    param: data: the dataset containing the epochs to extract features from
    param: df_sample: the dataframe containing the sampled participant IDs and their corresponding diagnosis
    param: num_extracted_features: the number of features to extract
    param: eval: boolean to evaluate the features
    param: to_disk: boolean to save the features to disk
    param: pretext_model: the model to extract features from
    param: weights_dir: the directory containing the weights of the pretext model
    """
    # Need to define model class here to avoid issues with different number of extracted features
    # create Conv2d with max norm constraint
    class Conv2dWithConstraint(nn.Conv2d):
        def __init__(self, *args, max_norm: int = 1, **kwargs):
            self.max_norm = max_norm
            super(Conv2dWithConstraint, self).__init__(*args, **kwargs)

        def forward(self, x: torch.Tensor) -> torch.Tensor:
            self.weight.data = torch.renorm(self.weight.data, p=2, dim=0, maxnorm=self.max_norm)
            return super(Conv2dWithConstraint, self).forward(x)
        
    class EEGNet(nn.Module):
        """
        Code taken and adjusted from pytorch implementation of EEGNet
        url: https://github.com/torcheeg/torcheeg/blob/v1.1.0/torcheeg/models/cnn/eegnet.py#L5
        """
        def __init__(self,
                    chunk_size: int = 1244, # number of data points in each EEG chunk
                    num_electrodes: int = 26, # number of EEG electrodes
                    F1: int = 8, # number of filters in first convolutional layer
                    F2: int = 16, # number of filters in second convolutional layer
                    D: int = 2, # depth multiplier
                    num_extracted_features: int = num_extracted_features, # number of features to extract
                    kernel_1: int = 64, # the filter size of block 1 (half of sfreq (125 Hz))
                    kernel_2: int = 16, # the filter size of block 2 (one eight of sfreq (500 Hz))
                    dropout: float = 0.25): # dropout rate
            super(EEGNet, self).__init__()
            self.F1 = F1
            self.F2 = F2
            self.D = D
            self.chunk_size = chunk_size
            self.num_extracted_features = num_extracted_features
            self.num_electrodes = num_electrodes
            self.kernel_1 = kernel_1
            self.kernel_2 = kernel_2
            self.dropout = dropout

            self.block1 = nn.Sequential(
                nn.Conv2d(1, self.F1, (1, self.kernel_1), stride=1, padding=(0, self.kernel_1 // 2), bias=False),
                nn.BatchNorm2d(self.F1, momentum=0.01, affine=True, eps=1e-3),
                Conv2dWithConstraint(self.F1,
                                    self.F1 * self.D, (self.num_electrodes, 1),
                                    max_norm=1,
                                    stride=1,
                                    padding=(0, 0),
                                    groups=self.F1,
                                    bias=False), nn.BatchNorm2d(self.F1 * self.D, momentum=0.01, affine=True, eps=1e-3),
                nn.ELU(), nn.AvgPool2d((1, 4), stride=4), nn.Dropout(p=dropout))

            self.block2 = nn.Sequential(
                nn.Conv2d(self.F1 * self.D,
                        self.F1 * self.D, (1, self.kernel_2),
                        stride=1,
                        padding=(0, self.kernel_2 // 2),
                        bias=False,
                        groups=self.F1 * self.D),
                nn.Conv2d(self.F1 * self.D, self.F2, 1, padding=(0, 0), groups=1, bias=False, stride=1),
                nn.BatchNorm2d(self.F2, momentum=0.01, affine=True, eps=1e-3), nn.ELU(), nn.AvgPool2d((1, 8), stride=8),
                nn.Dropout(p=dropout))

            self.lin = nn.Linear(self.feature_dim(), num_extracted_features, bias=False)


        def feature_dim(self):
            # function to calculate the number of features after the convolutional blocks
            with torch.no_grad():
                mock_eeg = torch.zeros(1, 1, self.num_electrodes, self.chunk_size)

                mock_eeg = self.block1(mock_eeg)
                mock_eeg = self.block2(mock_eeg)

            return self.F2 * mock_eeg.shape[3]

        def forward(self, x: torch.Tensor) -> torch.Tensor:
            x = self.block1(x)
            x = self.block2(x)
            x = x.flatten(start_dim=1)
            x = self.lin(x)
            return x
    
    class ShallowNet(nn.Module):
        """
        Pytorch implementation of the ShallowNet Encoder.
        Code taken and adjusted from:
        https://github.com/MedMaxLab/selfEEG/blob/024402ba4bde95051d86ab2524cc71105bfd5c25/selfeeg/models/zoo.py#L693
        """

        def __init__(self,
                    samples=1244,
                    chans=26, # number of EEG channels
                    F=40, # number of output filters in the temporal convolution layer
                    K1=25, # length of the temporal convolutional layer
                    pool=75, # temporal pooling kernel size
                    dropout=0.2, # dropout probability
                    num_extracted_features=num_extracted_features # number of features to extract
                    ):

            super(ShallowNet, self).__init__()
            self.conv1 = nn.Conv2d(1, F, (1, K1), stride=(1, 1))
            self.conv2 = nn.Conv2d(F, F, (chans, 1), stride=(1, 1))
            self.batch1 = nn.BatchNorm2d(F)
            self.pool2 = nn.AvgPool2d((1, pool), stride=(1, 15))
            self.flatten2 = nn.Flatten()
            self.drop1 = nn.Dropout(dropout)
            self.lin = nn.Linear(
                F * ((samples - K1 + 1 - pool) // 15 + 1), num_extracted_features
            )

        def forward(self, x):
            x = self.conv1(x)
            x = self.conv2(x)
            x = self.batch1(x)
            x = torch.square(x)
            x = self.pool2(x)
            x = torch.log(torch.clamp(x, 1e-7, 10000))
            x = self.flatten2(x)
            x = self.drop1(x)
            x = self.lin(x)

            return x
    
    if pretext_model == 'EEGNet':
        pretext_model = EEGNet()
    if pretext_model == 'ShallowNet':
        pretext_model = ShallowNet()
        
    pretrained_weights = torch.load(f'{weights_dir}\{pretext_task}_weights.pt')
    pretrained_model = transfer_weights(pretrained_weights, pretext_model)
    features_df = extract_features(pretrained_model, data, pretext_task, df_sample=df_sample, to_disk=to_disk)
    if eval:
        evaluate_features(features_df)
    
    return

### randomly initialized model

In [7]:
features_df = extract_features(EEGNet(), dataset, df_sample=df_sample, pretext_task='random', to_disk=False)
evaluate_features(features_df)

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-0.025231,0.558265,0.386430,-0.427448,-0.195549,-0.385354,0.035971,0.385304,0.106288,0.573986,...,0.261263,0.668248,-0.518208,0.242272,0.186938,0.214983,0.118263,-0.321973,sub-87964717,SMC
1,-0.111984,0.977839,0.131731,-0.051678,-0.290045,-0.434035,0.237411,-0.101681,-0.275212,0.762277,...,0.520117,0.624576,-0.485054,0.251544,-0.037080,-0.125105,-0.325259,-0.672581,sub-87964717,SMC
2,-0.033515,0.690026,-0.037421,-0.427968,-0.044509,-0.273484,0.223849,0.226518,0.001657,0.772421,...,0.178688,0.269243,-0.326347,-0.047144,-0.056313,0.331418,0.099657,-0.459115,sub-87964717,SMC


quick SVM model
[[58 17  5 16 12]
 [15 36  9 32 16]
 [13 29 24 21 21]
 [ 0 27 21 33 27]
 [ 3  4 15 26 60]]
              precision    recall  f1-score   support

        ADHD       0.65      0.54      0.59       108
     HEALTHY       0.32      0.33      0.33       108
         MDD       0.32      0.22      0.26       108
         OCD       0.26      0.31      0.28       108
         SMC       0.44      0.56      0.49       108

    accuracy                           0.39       540
   macro avg       0.40      0.39      0.39       540
weighted avg       0.40      0.39      0.39       540

0.3899649803774411

quick random forest model
[[62 14 14 10  8]
 [18 34 20 22 14]
 [22 28 26 15 17]
 [10 24 28 18 28]
 [ 3 23 15 19 48]]
              precision    recall  f1-score   support

        ADHD       0.54      0.57      0.56       108
     HEALTHY       0.28      0.31      0.29       108
         MDD       0.25      0.24      0.25       108
         OCD       0.21      0.17      0.19       

### pretext model with default parameters (0.25 dropout, 0 weight decay, binary cross entropy loss)

In [7]:
# best model checkpoint
get_ssl_features('acrosssub_default_pretext_model', dataset, df_sample, eval=True, to_disk=True)

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,3.623887,-1.228523,-1.482568,1.651297,-17.790987,5.814167,-1.637995,7.890889,0.264064,-1.745725,...,1.176101,19.422047,1.221881,9.082914,10.107931,1.013603,-0.151076,-0.010935,sub-87964717,SMC
1,4.063910,-0.249095,-1.333639,0.852530,-16.195347,5.602299,-0.927352,6.921437,-0.603323,-3.448514,...,0.694274,17.927160,2.676285,9.526098,10.338356,1.623796,0.002702,0.939451,sub-87964717,SMC
2,3.290289,0.072605,-0.576546,1.302655,-16.728363,5.784608,-1.060832,6.392365,1.129892,-3.624918,...,1.223866,19.308262,-0.159160,8.258363,11.810682,3.635775,0.800181,1.143464,sub-87964717,SMC


quick SVM model
[[44  9 11 30 14]
 [12 40 11 33 12]
 [16  6 22 24 40]
 [14 33 14 27 20]
 [18  4 31  3 52]]
              precision    recall  f1-score   support

        ADHD       0.42      0.41      0.42       108
     HEALTHY       0.43      0.37      0.40       108
         MDD       0.25      0.20      0.22       108
         OCD       0.23      0.25      0.24       108
         SMC       0.38      0.48      0.42       108

    accuracy                           0.34       540
   macro avg       0.34      0.34      0.34       540
weighted avg       0.34      0.34      0.34       540

0.34024176421440494

quick random forest model
[[63  0 23 12 10]
 [ 2 52 15 29 10]
 [10 12 24 35 27]
 [ 6 30 20 23 29]
 [15  7 16  7 63]]
              precision    recall  f1-score   support

        ADHD       0.66      0.58      0.62       108
     HEALTHY       0.51      0.48      0.50       108
         MDD       0.24      0.22      0.23       108
         OCD       0.22      0.21      0.21      

In [8]:
# overtrained model
get_ssl_features('overtrained_acrosssub_default_pretext_model', dataset, df_sample, eval=True, to_disk=True)

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,4.888031,-1.091719,-1.842990,1.659128,-18.793324,7.022872,-1.732366,9.151362,0.017521,-0.975390,...,0.738329,20.309301,1.319816,10.909989,9.268280,-0.507095,-0.809679,0.530718,sub-87964717,SMC
1,5.392010,-0.476683,-1.306184,-0.017972,-17.125385,7.146060,-0.269107,7.905194,-0.789516,-3.939141,...,-0.030765,18.537172,3.245900,11.278497,10.519344,0.110141,-0.592927,1.838493,sub-87964717,SMC
2,3.965634,0.235488,-0.463260,0.626689,-17.413103,6.744870,-1.186025,7.509469,0.890550,-3.323798,...,0.611838,19.963472,0.414842,10.219461,11.495079,3.069146,-0.335685,2.482077,sub-87964717,SMC


quick SVM model
[[44  8  6 34 16]
 [19 36  9 28 16]
 [15  5 22 21 45]
 [ 6 28 17 37 20]
 [16  1 27  9 55]]
              precision    recall  f1-score   support

        ADHD       0.44      0.41      0.42       108
     HEALTHY       0.46      0.33      0.39       108
         MDD       0.27      0.20      0.23       108
         OCD       0.29      0.34      0.31       108
         SMC       0.36      0.51      0.42       108

    accuracy                           0.36       540
   macro avg       0.36      0.36      0.36       540
weighted avg       0.36      0.36      0.36       540

0.3556582280142917

quick random forest model
[[63  0 16 17 12]
 [ 3 61 17 18  9]
 [ 9  2 31 30 36]
 [ 3 25 18 29 33]
 [18  4 18  5 63]]
              precision    recall  f1-score   support

        ADHD       0.66      0.58      0.62       108
     HEALTHY       0.66      0.56      0.61       108
         MDD       0.31      0.29      0.30       108
         OCD       0.29      0.27      0.28       

### pretext ShallowNet with default parameters trained for 3 epochs (0.25 dropout, 0 weight decay, binary cross entropy loss)

In [6]:
# best model checkpoint
get_ssl_features('acrosssub_shallownet_default_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-5.687540,-6.149271,3.001966,-3.595043,-0.623438,1.749660,-0.526338,-0.606515,-1.142817,2.034628,...,4.798987,-0.836588,-0.427841,-0.229442,-4.803131,-2.258094,-3.364549,2.675598,sub-87964717,SMC
1,-4.606072,-5.812068,-3.228870,-3.513926,-4.965502,2.462229,0.668698,-0.366934,0.487594,2.563390,...,5.277506,-0.330681,1.816657,2.663601,-3.906834,-2.400312,-5.455590,3.717802,sub-87964717,SMC
2,-4.881678,-5.581199,0.875168,-3.299997,-0.959724,0.015956,-4.779692,-3.217856,-2.067912,1.177492,...,5.249906,-0.571930,1.289420,2.425325,-4.904214,-2.458901,-4.525682,4.109240,sub-87964717,SMC


quick SVM model
[[57 15 14 18  4]
 [ 6 58 24 16  4]
 [20 21 40 12 15]
 [ 6 44 21 16 21]
 [16  5 24  1 62]]
              precision    recall  f1-score   support

        ADHD       0.54      0.53      0.54       108
     HEALTHY       0.41      0.54      0.46       108
         MDD       0.33      0.37      0.35       108
         OCD       0.25      0.15      0.19       108
         SMC       0.58      0.57      0.58       108

    accuracy                           0.43       540
   macro avg       0.42      0.43      0.42       540
weighted avg       0.42      0.43      0.42       540

0.4220513527217432

quick random forest model
[[60  7 23 13  5]
 [ 6 51 18 27  6]
 [26 18 35 11 18]
 [11 24 33 20 20]
 [12  5 23  6 62]]
              precision    recall  f1-score   support

        ADHD       0.52      0.56      0.54       108
     HEALTHY       0.49      0.47      0.48       108
         MDD       0.27      0.32      0.29       108
         OCD       0.26      0.19      0.22       

In [7]:
# overtrained model
get_ssl_features('overtrained_acrosssub_shallownet_default_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-5.687540,-6.149271,3.001966,-3.595043,-0.623438,1.749660,-0.526338,-0.606515,-1.142817,2.034628,...,4.798987,-0.836588,-0.427841,-0.229442,-4.803131,-2.258094,-3.364549,2.675598,sub-87964717,SMC
1,-4.606072,-5.812068,-3.228870,-3.513926,-4.965502,2.462229,0.668698,-0.366934,0.487594,2.563390,...,5.277506,-0.330681,1.816657,2.663601,-3.906834,-2.400312,-5.455590,3.717802,sub-87964717,SMC
2,-4.881678,-5.581199,0.875168,-3.299997,-0.959724,0.015956,-4.779692,-3.217856,-2.067912,1.177492,...,5.249906,-0.571930,1.289420,2.425325,-4.904214,-2.458901,-4.525682,4.109240,sub-87964717,SMC


quick SVM model
[[57 15 14 18  4]
 [ 6 58 24 16  4]
 [20 21 40 12 15]
 [ 6 44 21 16 21]
 [16  5 24  1 62]]
              precision    recall  f1-score   support

        ADHD       0.54      0.53      0.54       108
     HEALTHY       0.41      0.54      0.46       108
         MDD       0.33      0.37      0.35       108
         OCD       0.25      0.15      0.19       108
         SMC       0.58      0.57      0.58       108

    accuracy                           0.43       540
   macro avg       0.42      0.43      0.42       540
weighted avg       0.42      0.43      0.42       540

0.4220513527217432

quick random forest model
[[59  6 27 11  5]
 [ 8 46 17 29  8]
 [25 18 35 12 18]
 [12 21 38 20 17]
 [10  7 26  3 62]]
              precision    recall  f1-score   support

        ADHD       0.52      0.55      0.53       108
     HEALTHY       0.47      0.43      0.45       108
         MDD       0.24      0.32      0.28       108
         OCD       0.27      0.19      0.22       

### pretext ShallowNet with default parameters trained for 10 epochs (0.25 dropout, 0 weight decay, binary cross entropy loss)

In [8]:
# best model checkpoint
get_ssl_features('acrosssub_more_ep_shallownet_default_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-4.189551,0.056380,0.080604,11.732378,3.034628,0.794952,1.408191,-7.797366,0.980474,2.151414,...,5.338130,-9.361512,0.508444,1.020527,-7.753326,0.120419,-0.304918,5.474427,sub-87964717,SMC
1,-2.195098,-0.793652,-0.669229,8.055529,-0.472367,2.598654,1.083506,-10.556555,-0.082156,1.687747,...,6.801520,-13.148778,-0.204826,-0.730834,-4.019993,-0.442744,-1.361034,7.417520,sub-87964717,SMC
2,-3.497191,0.740369,1.286040,7.856913,4.611308,1.576089,2.245129,-6.196683,-0.283627,2.566361,...,4.506096,-5.089340,-0.700921,4.147587,-6.219935,0.815815,1.608547,6.667706,sub-87964717,SMC


quick SVM model
[[58  6 14 26  4]
 [ 4 66 16 20  2]
 [16 15 36 22 19]
 [ 3 46 34 21  4]
 [14 14 12 12 56]]
              precision    recall  f1-score   support

        ADHD       0.61      0.54      0.57       108
     HEALTHY       0.45      0.61      0.52       108
         MDD       0.32      0.33      0.33       108
         OCD       0.21      0.19      0.20       108
         SMC       0.66      0.52      0.58       108

    accuracy                           0.44       540
   macro avg       0.45      0.44      0.44       540
weighted avg       0.45      0.44      0.44       540

0.4395232352305773

quick random forest model
[[61  6 10 23  8]
 [ 5 57 14 30  2]
 [16 19 25 25 23]
 [ 0 38 41 19 10]
 [ 7 19 13  7 62]]
              precision    recall  f1-score   support

        ADHD       0.69      0.56      0.62       108
     HEALTHY       0.41      0.53      0.46       108
         MDD       0.24      0.23      0.24       108
         OCD       0.18      0.18      0.18       

In [9]:
# overtrained model
get_ssl_features('overtrained_acrosssub_more_ep_shallownet_default_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-3.799145,-3.209580,-1.015182,13.616658,-3.772485,0.425463,4.280510,-14.531560,-3.346121,-3.315176,...,6.597253,-7.583640,1.451107,0.802926,-6.216964,10.980362,-4.946252,8.696836,sub-87964717,SMC
1,-1.555926,-3.220124,-1.553759,11.932609,-5.865590,2.185759,5.390312,-16.609692,-2.232033,-3.740904,...,8.174911,-6.722993,-1.228109,-6.023156,-1.735186,8.771534,-3.315291,10.725916,sub-87964717,SMC
2,-1.925291,-2.443768,3.334188,11.736165,-4.187596,0.168126,6.149526,-14.142980,-3.659545,-0.820885,...,4.459185,-1.050718,-0.615487,0.882863,-4.807421,10.799395,-1.731158,8.928881,sub-87964717,SMC


quick SVM model
[[58  9 16 22  3]
 [ 8 69 10 20  1]
 [16 22 27 25 18]
 [ 6 46 18 30  8]
 [16 11 25 10 46]]
              precision    recall  f1-score   support

        ADHD       0.56      0.54      0.55       108
     HEALTHY       0.44      0.64      0.52       108
         MDD       0.28      0.25      0.26       108
         OCD       0.28      0.28      0.28       108
         SMC       0.61      0.43      0.50       108

    accuracy                           0.43       540
   macro avg       0.43      0.43      0.42       540
weighted avg       0.43      0.43      0.42       540

0.4223400356193377

quick random forest model
[[61  7 17 22  1]
 [ 7 70 10 16  5]
 [13 28 28 21 18]
 [ 0 42 32 21 13]
 [11 13 17  5 62]]
              precision    recall  f1-score   support

        ADHD       0.66      0.56      0.61       108
     HEALTHY       0.44      0.65      0.52       108
         MDD       0.27      0.26      0.26       108
         OCD       0.25      0.19      0.22       

### pretext model with (0.5 dropout, 0.01 weight decay, binary cross entropy loss) trained for 50 epochs

In [8]:
# best model checkpoint
get_ssl_features('acrosssub_0.5dropout_0.01wd_pretext_model', dataset, df_sample, eval=True, to_disk=True)

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,0.110355,0.811385,0.972408,1.268003,0.849162,-1.444711,-0.000281,-0.144839,-0.278361,0.770080,...,1.697409,-0.454907,0.853184,1.153516,-0.163238,1.132012,-1.542204,-0.077861,sub-87964717,SMC
1,-0.032289,0.799907,0.899556,1.262478,0.876555,-1.422669,-0.066222,0.037951,-0.280847,0.757096,...,1.693310,-0.431586,0.814568,1.041446,0.013759,1.129061,-1.500744,0.015565,sub-87964717,SMC
2,0.321387,0.773045,0.829811,1.196555,0.767482,-1.333442,-0.035082,-0.103440,-0.247684,0.693844,...,1.487311,-0.390890,0.752712,0.992581,-0.097055,0.994175,-1.416777,0.161192,sub-87964717,SMC


quick SVM model
[[55 12  0 31 10]
 [11 16  1 42 38]
 [16 21  0 28 43]
 [ 0 34  3 32 39]
 [ 0 12  2 49 45]]
              precision    recall  f1-score   support

        ADHD       0.67      0.51      0.58       108
     HEALTHY       0.17      0.15      0.16       108
         MDD       0.00      0.00      0.00       108
         OCD       0.18      0.30      0.22       108
         SMC       0.26      0.42      0.32       108

    accuracy                           0.27       540
   macro avg       0.25      0.27      0.26       540
weighted avg       0.25      0.27      0.26       540

0.25505873859743794

quick random forest model
[[54 17  8 24  5]
 [15 35 17 21 20]
 [24 26 25 20 13]
 [ 2 37 26 23 20]
 [ 3 20 20 47 18]]
              precision    recall  f1-score   support

        ADHD       0.55      0.50      0.52       108
     HEALTHY       0.26      0.32      0.29       108
         MDD       0.26      0.23      0.25       108
         OCD       0.17      0.21      0.19      

In [10]:
# overtrained model
get_ssl_features('overtrained_acrosssub_0.5dropout_0.01wd_pretext_model', dataset, df_sample, eval=True, to_disk=True)

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,0.110355,0.811385,0.972408,1.268003,0.849162,-1.444711,-0.000281,-0.144839,-0.278361,0.770080,...,1.697409,-0.454907,0.853184,1.153516,-0.163238,1.132012,-1.542204,-0.077861,sub-87964717,SMC
1,-0.032289,0.799907,0.899556,1.262478,0.876555,-1.422669,-0.066222,0.037951,-0.280847,0.757096,...,1.693310,-0.431586,0.814568,1.041446,0.013759,1.129061,-1.500744,0.015565,sub-87964717,SMC
2,0.321387,0.773045,0.829811,1.196555,0.767482,-1.333442,-0.035082,-0.103440,-0.247684,0.693844,...,1.487311,-0.390890,0.752712,0.992581,-0.097055,0.994175,-1.416777,0.161192,sub-87964717,SMC


quick SVM model
[[55 12  0 31 10]
 [11 16  1 42 38]
 [16 21  0 28 43]
 [ 0 34  3 32 39]
 [ 0 12  2 49 45]]
              precision    recall  f1-score   support

        ADHD       0.67      0.51      0.58       108
     HEALTHY       0.17      0.15      0.16       108
         MDD       0.00      0.00      0.00       108
         OCD       0.18      0.30      0.22       108
         SMC       0.26      0.42      0.32       108

    accuracy                           0.27       540
   macro avg       0.25      0.27      0.26       540
weighted avg       0.25      0.27      0.26       540

0.25505873859743794

quick random forest model
[[57 15  7 25  4]
 [17 32 16 21 22]
 [25 20 25 17 21]
 [ 3 39 26 24 16]
 [ 6 23 15 41 23]]
              precision    recall  f1-score   support

        ADHD       0.53      0.53      0.53       108
     HEALTHY       0.25      0.30      0.27       108
         MDD       0.28      0.23      0.25       108
         OCD       0.19      0.22      0.20      

### pretext ShallowNet with (0.5 dropout, 0.01 weight decay, binary cross entropy loss)

In [6]:
# best model checkpoint
get_ssl_features('acrosssub_0.5do_0.01wd_shallownet_default_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-0.213381,1.492459,6.011405,0.315197,1.038163,-1.462611,-5.850992,0.775347,-2.554164,-3.245762,...,-0.543933,-0.982938,0.676638,-0.229126,-3.363280,0.424538,-2.649733,-0.657113,sub-87964717,SMC
1,-0.482036,-0.692106,4.668519,0.076917,0.673403,-1.346192,-5.922452,1.063697,-2.927278,-2.553437,...,-0.079214,-0.303469,-0.057817,-0.737458,-3.391969,1.267721,-2.026364,-0.064307,sub-87964717,SMC
2,-1.576242,0.987797,6.256711,0.852646,0.057229,-2.786604,-5.047675,0.760222,-4.281447,-3.566510,...,0.297048,-1.026178,0.995004,-0.406680,-2.985252,1.212061,-1.645706,0.593830,sub-87964717,SMC


quick SVM model
[[56 17  5 25  5]
 [ 0 62  7 30  9]
 [15 20 16 28 29]
 [ 0 64 11 30  3]
 [ 9 46 20  6 27]]
              precision    recall  f1-score   support

        ADHD       0.70      0.52      0.60       108
     HEALTHY       0.30      0.57      0.39       108
         MDD       0.27      0.15      0.19       108
         OCD       0.25      0.28      0.26       108
         SMC       0.37      0.25      0.30       108

    accuracy                           0.35       540
   macro avg       0.38      0.35      0.35       540
weighted avg       0.38      0.35      0.35       540

0.3482376723600712

quick random forest model
[[67  5 11 20  5]
 [ 6 55  9 20 18]
 [11  9 22 33 33]
 [14 41 10 35  8]
 [20 16 15 30 27]]
              precision    recall  f1-score   support

        ADHD       0.57      0.62      0.59       108
     HEALTHY       0.44      0.51      0.47       108
         MDD       0.33      0.20      0.25       108
         OCD       0.25      0.32      0.28       

In [8]:
# overtrained model
get_ssl_features('overtrained_acrosssub_0.5do_0.01wd_shallownet_default_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-1.956640,3.399338,20.473583,4.262378,-1.058155,-1.775384,1.953410,0.986932,-21.866470,2.097848,...,2.351862,3.334328,1.462563,1.250490,0.084261,1.905367,6.059560,2.317672,sub-87964717,SMC
1,-1.890188,2.957329,17.483562,4.950645,0.233775,-2.944295,1.779520,2.352410,-23.581444,1.400910,...,1.756081,2.653468,1.080964,2.052077,0.112226,1.602259,4.607489,1.694117,sub-87964717,SMC
2,-1.830626,3.425448,20.148228,4.462543,-1.041958,-2.935012,2.063508,1.247084,-23.540009,1.278066,...,1.648017,3.042125,0.888284,1.202089,0.104508,1.905920,5.442610,1.505053,sub-87964717,SMC


quick SVM model
[[60 12  8 24  4]
 [14 55  6 29  4]
 [14 14 30 15 35]
 [ 0 27 31 50  0]
 [12  3 15 21 57]]
              precision    recall  f1-score   support

        ADHD       0.60      0.56      0.58       108
     HEALTHY       0.50      0.51      0.50       108
         MDD       0.33      0.28      0.30       108
         OCD       0.36      0.46      0.40       108
         SMC       0.57      0.53      0.55       108

    accuracy                           0.47       540
   macro avg       0.47      0.47      0.47       540
weighted avg       0.47      0.47      0.47       540

0.4670343415296552

quick random forest model
[[57  2  8 33  8]
 [13 42 18 31  4]
 [13 29 31 11 24]
 [ 8 34 20 33 13]
 [16 10 24 15 43]]
              precision    recall  f1-score   support

        ADHD       0.53      0.53      0.53       108
     HEALTHY       0.36      0.39      0.37       108
         MDD       0.31      0.29      0.30       108
         OCD       0.27      0.31      0.29       

### pretext ShallowNet with (0.5 dropout, 0 weight decay, binary cross entropy loss)

In [9]:
# best model checkpoint
get_ssl_features('acrosssub_0.5do_0wd_shallownet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-7.407146,0.327229,-6.790859,2.743637,9.213750,37.415627,8.782216,7.385911,8.118476,-28.716213,...,-1.505764,4.080109,-6.691596,3.842114,-13.304543,21.997303,-3.540983,-0.138042,sub-87964717,SMC
1,-8.238667,-0.067489,0.893605,0.914843,10.331154,32.996189,9.946080,1.844107,9.314865,-28.173861,...,4.093190,0.338762,-4.100086,5.083673,-12.276780,27.578815,-1.246826,0.694709,sub-87964717,SMC
2,-3.488221,0.959020,-3.741003,1.737046,10.633974,33.116806,1.553113,4.192796,5.682030,-26.670061,...,0.769521,2.382316,-9.240067,5.146344,-12.845651,24.224911,-3.833890,-0.033360,sub-87964717,SMC


quick SVM model
[[55  9 16 27  1]
 [14 57 17 15  5]
 [19 20 12 40 17]
 [ 0 54 23 21 10]
 [ 8  9 11  9 71]]
              precision    recall  f1-score   support

        ADHD       0.57      0.51      0.54       108
     HEALTHY       0.38      0.53      0.44       108
         MDD       0.15      0.11      0.13       108
         OCD       0.19      0.19      0.19       108
         SMC       0.68      0.66      0.67       108

    accuracy                           0.40       540
   macro avg       0.40      0.40      0.39       540
weighted avg       0.40      0.40      0.39       540

0.3943716220929175

quick random forest model
[[64 13 18 12  1]
 [ 3 54  6 44  1]
 [23 21 14 26 24]
 [ 1 52 35 14  6]
 [11 11  7  7 72]]
              precision    recall  f1-score   support

        ADHD       0.63      0.59      0.61       108
     HEALTHY       0.36      0.50      0.42       108
         MDD       0.17      0.13      0.15       108
         OCD       0.14      0.13      0.13       

In [10]:
# overtrained model
get_ssl_features('acrosssub_0.5do_0wd_shallownet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-7.407146,0.327229,-6.790859,2.743637,9.213750,37.415627,8.782216,7.385911,8.118476,-28.716213,...,-1.505764,4.080109,-6.691596,3.842114,-13.304543,21.997303,-3.540983,-0.138042,sub-87964717,SMC
1,-8.238667,-0.067489,0.893605,0.914843,10.331154,32.996189,9.946080,1.844107,9.314865,-28.173861,...,4.093190,0.338762,-4.100086,5.083673,-12.276780,27.578815,-1.246826,0.694709,sub-87964717,SMC
2,-3.488221,0.959020,-3.741003,1.737046,10.633974,33.116806,1.553113,4.192796,5.682030,-26.670061,...,0.769521,2.382316,-9.240067,5.146344,-12.845651,24.224911,-3.833890,-0.033360,sub-87964717,SMC


quick SVM model
[[55  9 16 27  1]
 [14 57 17 15  5]
 [19 20 12 40 17]
 [ 0 54 23 21 10]
 [ 8  9 11  9 71]]
              precision    recall  f1-score   support

        ADHD       0.57      0.51      0.54       108
     HEALTHY       0.38      0.53      0.44       108
         MDD       0.15      0.11      0.13       108
         OCD       0.19      0.19      0.19       108
         SMC       0.68      0.66      0.67       108

    accuracy                           0.40       540
   macro avg       0.40      0.40      0.39       540
weighted avg       0.40      0.40      0.39       540

0.3943716220929175

quick random forest model
[[63 10 23 12  0]
 [ 1 56  6 44  1]
 [23 19 13 30 23]
 [ 0 47 39 14  8]
 [10 10  8  7 73]]
              precision    recall  f1-score   support

        ADHD       0.65      0.58      0.61       108
     HEALTHY       0.39      0.52      0.45       108
         MDD       0.15      0.12      0.13       108
         OCD       0.13      0.13      0.13       

### pretext model with (0.5 dropout, no weight decay, binary cross entropy loss) trained for 50 epochs

In [6]:
# best model checkpoint
get_ssl_features('acrosssub_0.5dropout_0wd_pretext_model', dataset, df_sample, eval=True, to_disk=True)

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-0.307977,-0.215065,-0.138612,0.188878,-1.927320,0.024882,0.133953,0.026465,0.280698,-0.142190,...,-0.391678,0.225190,-1.955430,0.004394,-0.021402,0.032482,0.127371,2.018096,sub-87964717,SMC
1,-0.053138,0.493606,-0.206609,-0.321768,-1.964830,0.097457,0.094318,0.048917,-0.240670,-0.373819,...,0.025949,-0.074846,-1.882266,0.063315,0.223343,0.087265,-0.328646,2.161488,sub-87964717,SMC
2,0.195065,0.087239,0.145831,-0.213409,-1.759792,0.002818,0.084725,-0.131807,-0.053811,-0.316187,...,-0.407896,0.031429,-1.778772,0.297457,-0.137110,0.097792,-0.445519,1.680954,sub-87964717,SMC


quick SVM model
[[57 32  0 14  5]
 [11 35  0 25 37]
 [16 33  0 13 46]
 [ 0 37  1 21 49]
 [ 0 33  0 34 41]]
              precision    recall  f1-score   support

        ADHD       0.68      0.53      0.59       108
     HEALTHY       0.21      0.32      0.25       108
         MDD       0.00      0.00      0.00       108
         OCD       0.20      0.19      0.20       108
         SMC       0.23      0.38      0.29       108

    accuracy                           0.29       540
   macro avg       0.26      0.29      0.27       540
weighted avg       0.26      0.29      0.27       540

0.26552213701473365

quick random forest model
[[58 16 15 14  5]
 [13 37 15 17 26]
 [19 17 18 31 23]
 [ 7 36 21 31 13]
 [ 5 23 14 46 20]]
              precision    recall  f1-score   support

        ADHD       0.57      0.54      0.55       108
     HEALTHY       0.29      0.34      0.31       108
         MDD       0.22      0.17      0.19       108
         OCD       0.22      0.29      0.25      

In [7]:
# overtrained model
get_ssl_features('overtrained_acrosssub_0.5dropout_0wd_pretext_model', dataset, df_sample, eval=True, to_disk=True)

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-0.294242,-0.195981,-0.143998,0.187191,-1.928485,0.032276,0.130113,0.027405,0.279228,-0.152299,...,-0.394884,0.216510,-1.957939,0.007444,-0.023034,0.035616,0.120691,2.011622,sub-87964717,SMC
1,-0.057847,0.489228,-0.206049,-0.321633,-1.970993,0.111270,0.085086,0.063148,-0.252367,-0.384709,...,0.022323,-0.076969,-1.890000,0.069849,0.225874,0.081027,-0.332357,2.160780,sub-87964717,SMC
2,0.205913,0.089601,0.142711,-0.220439,-1.766963,0.005877,0.080310,-0.132771,-0.060384,-0.333385,...,-0.407434,0.026319,-1.784355,0.293689,-0.147704,0.093849,-0.450984,1.689083,sub-87964717,SMC


quick SVM model
[[57 33  0 13  5]
 [11 36  0 24 37]
 [16 33  0 13 46]
 [ 0 38  0 19 51]
 [ 0 34  0 33 41]]
              precision    recall  f1-score   support

        ADHD       0.68      0.53      0.59       108
     HEALTHY       0.21      0.33      0.26       108
         MDD       0.00      0.00      0.00       108
         OCD       0.19      0.18      0.18       108
         SMC       0.23      0.38      0.28       108

    accuracy                           0.28       540
   macro avg       0.26      0.28      0.26       540
weighted avg       0.26      0.28      0.26       540

0.26294875042215465



c:\Users\tuur\anaconda3\envs\mne\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tuur\anaconda3\envs\mne\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tuur\anaconda3\envs\mne\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


quick random forest model
[[58 11 24 10  5]
 [17 37 16 18 20]
 [20 20 21 23 24]
 [ 6 38 19 26 19]
 [ 2 24 16 43 23]]
              precision    recall  f1-score   support

        ADHD       0.56      0.54      0.55       108
     HEALTHY       0.28      0.34      0.31       108
         MDD       0.22      0.19      0.21       108
         OCD       0.22      0.24      0.23       108
         SMC       0.25      0.21      0.23       108

    accuracy                           0.31       540
   macro avg       0.31      0.31      0.31       540
weighted avg       0.31      0.31      0.31       540

0.3051591420395


### pretext model with (0.25 dropout, 0.01 weight decay, binary cross entropy loss) trained for 50 epochs

In [8]:
# best model checkpoint
get_ssl_features('acrosssub_0.25dropout_0.01wd_pretext_model', dataset, df_sample, eval=True, to_disk=True)

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,0.126123,-0.207513,1.200154,0.027053,0.844097,-0.224665,-0.182237,0.230969,0.348675,-0.156983,...,-0.893586,1.371443,0.049336,0.043378,-0.018445,1.370729,0.093330,0.161098,sub-87964717,SMC
1,-0.059315,0.065023,1.228098,0.008292,0.819910,-0.207241,0.040250,-0.072927,0.336737,0.009810,...,-0.868467,1.302506,0.019837,0.014965,0.103793,1.255290,0.049264,-0.127574,sub-87964717,SMC
2,0.291039,-0.465372,1.202179,0.353446,0.802828,0.051728,-0.199987,-0.059852,0.027925,-0.151295,...,-0.801849,1.247910,-0.039929,0.039676,-0.260017,1.337238,0.032157,-0.562583,sub-87964717,SMC


quick SVM model
[[60 31  0 13  4]
 [11 44  0 25 28]
 [22 36  0 22 28]
 [ 0 40  0 29 39]
 [ 0 42  0 37 29]]
              precision    recall  f1-score   support

        ADHD       0.65      0.56      0.60       108
     HEALTHY       0.23      0.41      0.29       108
         MDD       0.00      0.00      0.00       108
         OCD       0.23      0.27      0.25       108
         SMC       0.23      0.27      0.25       108

    accuracy                           0.30       540
   macro avg       0.27      0.30      0.28       540
weighted avg       0.27      0.30      0.28       540

0.27659993781750003



c:\Users\tuur\anaconda3\envs\mne\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tuur\anaconda3\envs\mne\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tuur\anaconda3\envs\mne\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


quick random forest model
[[63 20  5 15  5]
 [13 32 21 27 15]
 [22 32 18 21 15]
 [ 4 34 26 18 26]
 [ 2 34 19 36 17]]
              precision    recall  f1-score   support

        ADHD       0.61      0.58      0.59       108
     HEALTHY       0.21      0.30      0.25       108
         MDD       0.20      0.17      0.18       108
         OCD       0.15      0.17      0.16       108
         SMC       0.22      0.16      0.18       108

    accuracy                           0.27       540
   macro avg       0.28      0.27      0.27       540
weighted avg       0.28      0.27      0.27       540

0.27320605689427113


In [9]:
# overtrained model
get_ssl_features('overtrained_acrosssub_0.25dropout_0.01wd_pretext_model', dataset, df_sample, eval=True, to_disk=True)

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,0.122879,-0.208553,1.189810,0.024261,0.832070,-0.223213,-0.179578,0.230247,0.347507,-0.157654,...,-0.880981,1.357200,0.052870,0.043568,-0.020767,1.354425,0.100774,0.163721,sub-87964717,SMC
1,-0.063217,0.070579,1.216221,0.003849,0.809047,-0.200670,0.040086,-0.069594,0.334794,0.014874,...,-0.858232,1.290785,0.019536,0.013853,0.102658,1.243865,0.055062,-0.129536,sub-87964717,SMC
2,0.285786,-0.463316,1.190094,0.349529,0.791883,0.056204,-0.194631,-0.062473,0.027171,-0.150910,...,-0.794846,1.237032,-0.041627,0.040274,-0.261118,1.322892,0.030744,-0.557040,sub-87964717,SMC


quick SVM model
[[60 32  0 13  3]
 [11 44  0 25 28]
 [21 37  0 22 28]
 [ 0 40  0 29 39]
 [ 0 43  0 36 29]]
              precision    recall  f1-score   support

        ADHD       0.65      0.56      0.60       108
     HEALTHY       0.22      0.41      0.29       108
         MDD       0.00      0.00      0.00       108
         OCD       0.23      0.27      0.25       108
         SMC       0.23      0.27      0.25       108

    accuracy                           0.30       540
   macro avg       0.27      0.30      0.28       540
weighted avg       0.27      0.30      0.28       540

0.2770418466950867



c:\Users\tuur\anaconda3\envs\mne\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tuur\anaconda3\envs\mne\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tuur\anaconda3\envs\mne\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


quick random forest model
[[62 19 10 11  6]
 [18 32 22 25 11]
 [21 31 15 26 15]
 [ 5 38 20 29 16]
 [ 6 28 17 42 15]]
              precision    recall  f1-score   support

        ADHD       0.55      0.57      0.56       108
     HEALTHY       0.22      0.30      0.25       108
         MDD       0.18      0.14      0.16       108
         OCD       0.22      0.27      0.24       108
         SMC       0.24      0.14      0.18       108

    accuracy                           0.28       540
   macro avg       0.28      0.28      0.28       540
weighted avg       0.28      0.28      0.28       540

0.2771977721085059


### pretext model with trained for 10 epochs

In [6]:
# best model checkpoint
get_ssl_features('acrosssub_10epochs_pretext_model', dataset, df_sample, eval=True, to_disk=True)

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,0.481440,0.633112,-1.726813,9.642133,0.692130,-10.080816,1.418434,1.354628,-3.802871,4.347657,...,2.109843,2.293432,4.100114,-5.626955,0.154212,1.305998,-1.614531,-7.114364,sub-87964717,SMC
1,-0.288733,0.207911,0.257113,6.740704,0.792730,-8.717131,0.794609,-1.659430,-2.662497,0.801926,...,-0.072879,2.290982,3.123032,-3.737323,0.073285,1.190753,-0.780681,-5.290376,sub-87964717,SMC
2,0.185403,0.507175,-0.284592,7.127849,0.205872,-10.026775,0.720594,-0.380045,-3.082051,1.919059,...,1.387761,2.440483,1.969689,-4.929652,0.935145,1.276471,-1.320437,-5.335428,sub-87964717,SMC


quick SVM model
[[56  3 29 12  8]
 [12 30 26 23 17]
 [15 21 27 16 29]
 [13 13 37 31 14]
 [15 14 13 17 49]]
              precision    recall  f1-score   support

        ADHD       0.50      0.52      0.51       108
     HEALTHY       0.37      0.28      0.32       108
         MDD       0.20      0.25      0.23       108
         OCD       0.31      0.29      0.30       108
         SMC       0.42      0.45      0.44       108

    accuracy                           0.36       540
   macro avg       0.36      0.36      0.36       540
weighted avg       0.36      0.36      0.36       540

0.35778966126851774

quick random forest model
[[64  1 24 10  9]
 [13 50  8 26 11]
 [20 20 26 22 20]
 [10 26 26 34 12]
 [16 19 19 18 36]]
              precision    recall  f1-score   support

        ADHD       0.52      0.59      0.55       108
     HEALTHY       0.43      0.46      0.45       108
         MDD       0.25      0.24      0.25       108
         OCD       0.31      0.31      0.31      

In [7]:
# overtrained model
get_ssl_features('overtrained_acrosssub_10epochs_pretext_model', dataset, df_sample, eval=True, to_disk=True)

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,0.193253,1.199918,0.319841,10.120821,0.304049,-8.346296,1.340301,0.298271,-3.376927,3.718822,...,1.891683,3.434391,2.526464,-3.987168,-0.804768,1.362534,-1.847329,-6.440199,sub-87964717,SMC
1,-0.553735,1.069052,2.289602,8.523880,0.626479,-8.482388,0.357972,-2.737444,-2.118034,0.706097,...,-1.176191,2.789662,2.244703,-3.112911,-1.079146,1.799031,-1.622424,-4.895182,sub-87964717,SMC
2,-0.051257,1.426289,1.955187,8.644217,0.076998,-9.104626,0.291343,-0.411044,-1.611388,1.955620,...,0.741185,2.922426,1.332080,-3.222737,-0.557499,1.172376,-1.824687,-4.568328,sub-87964717,SMC


quick SVM model
[[57  2 29 14  6]
 [13 33 24 25 13]
 [21 17 19 22 29]
 [ 7 16 38 33 14]
 [12 11 16 22 47]]
              precision    recall  f1-score   support

        ADHD       0.52      0.53      0.52       108
     HEALTHY       0.42      0.31      0.35       108
         MDD       0.15      0.18      0.16       108
         OCD       0.28      0.31      0.29       108
         SMC       0.43      0.44      0.43       108

    accuracy                           0.35       540
   macro avg       0.36      0.35      0.35       540
weighted avg       0.36      0.35      0.35       540

0.3532185398650852

quick random forest model
[[62  2 29 11  4]
 [12 45 11 29 11]
 [16 20 29 18 25]
 [14 28 20 32 14]
 [14 20 22 12 40]]
              precision    recall  f1-score   support

        ADHD       0.53      0.57      0.55       108
     HEALTHY       0.39      0.42      0.40       108
         MDD       0.26      0.27      0.26       108
         OCD       0.31      0.30      0.30       

### pretext model 150 features

In [8]:
# best model checkpoint
get_ssl_features('acrosssub_150feat_pretext_model', dataset, df_sample, eval=True, to_disk=True)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Documents\\Master_Data_Science\\Thesis\\thesis_code\\DataScience_Thesis\\data\\pretext_model_weights\\acrosssub_150feat_pretext_model_weights.pt'

In [10]:
# overtrained model
get_ssl_features('overtrained_acrosssub_150feat_pretext_model', dataset, df_sample, eval=True, to_disk=True, num_extracted_features=150)

features_df.shape = (2688, 152)


,0,1,2,3,4,5,6,7,8,9,...,142,143,144,145,146,147,148,149,ID,diagnosis
0,-2.559961,-3.054284,-2.644992,-6.277209,-0.028564,-0.906803,-21.773430,-0.134492,-1.485628,0.396045,...,1.410881,-1.888870,1.885950,-3.703908,-12.834751,2.617877,13.235068,-1.618843,sub-87964717,SMC
1,-0.068584,-0.779696,-0.959284,-9.155769,0.970993,-0.524335,-19.471977,2.176560,-1.005807,1.004661,...,4.837210,-2.676227,0.850829,-7.057852,-10.173094,1.023762,10.949565,-0.758843,sub-87964717,SMC
2,-0.333431,-1.506322,-3.900646,-7.290970,-0.305780,-2.440059,-18.541176,0.154866,-0.744295,1.786914,...,5.353173,-4.430460,1.096191,-5.224116,-8.479725,2.299004,10.076630,-1.119349,sub-87964717,SMC


quick SVM model
[[53  0 13 31 11]
 [ 7 59 19 15  8]
 [26  9 38 19 16]
 [ 9 35 38 21  5]
 [17 34 13 18 26]]
              precision    recall  f1-score   support

        ADHD       0.47      0.49      0.48       108
     HEALTHY       0.43      0.55      0.48       108
         MDD       0.31      0.35      0.33       108
         OCD       0.20      0.19      0.20       108
         SMC       0.39      0.24      0.30       108

    accuracy                           0.36       540
   macro avg       0.36      0.36      0.36       540
weighted avg       0.36      0.36      0.36       540

0.35845846927937236

quick random forest model
[[59  0 19 24  6]
 [ 7 64  3 29  5]
 [14 16 38 18 22]
 [12 33 33 28  2]
 [17 30  9 18 34]]
              precision    recall  f1-score   support

        ADHD       0.54      0.55      0.54       108
     HEALTHY       0.45      0.59      0.51       108
         MDD       0.37      0.35      0.36       108
         OCD       0.24      0.26      0.25      

### pretext ShallowNet with 150 features

In [7]:
# best model checkpoint
get_ssl_features('acrosssub_shallownet_150feat_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet', num_extracted_features=150)

features_df.shape = (2688, 152)


,0,1,2,3,4,5,6,7,8,9,...,142,143,144,145,146,147,148,149,ID,diagnosis
0,8.513660,-2.696927,-4.387354,2.608038,-32.631870,4.399547,-16.415943,0.529955,7.457929,3.418176,...,7.535275,14.255805,-3.302823,-10.527999,-3.135436,-6.404281,-3.802806,1.096164,sub-87964717,SMC
1,9.001474,-2.931648,-0.907542,2.912841,-28.491467,7.879901,-16.816526,6.210576,10.098440,2.137046,...,8.171915,13.152274,-2.158560,-12.107771,-2.322901,-0.709193,-5.222330,-0.211367,sub-87964717,SMC
2,9.597940,-2.530099,-4.957353,2.735745,-31.037685,5.652042,-19.736437,4.820051,10.401624,2.007243,...,7.093247,14.792567,-4.028124,-13.116168,-3.897216,-6.471945,-4.124007,2.688720,sub-87964717,SMC


quick SVM model
[[52  4 26 23  3]
 [ 6 64 12 23  3]
 [19  2 39 28 20]
 [ 0 42 40 19  7]
 [12  2 17 12 65]]
              precision    recall  f1-score   support

        ADHD       0.58      0.48      0.53       108
     HEALTHY       0.56      0.59      0.58       108
         MDD       0.29      0.36      0.32       108
         OCD       0.18      0.18      0.18       108
         SMC       0.66      0.60      0.63       108

    accuracy                           0.44       540
   macro avg       0.46      0.44      0.45       540
weighted avg       0.46      0.44      0.45       540

0.447256224984567

quick random forest model
[[64  3 21 20  0]
 [ 2 73 10 20  3]
 [19  8 34 28 19]
 [ 0 46 32 25  5]
 [ 4  6 22  7 69]]
              precision    recall  f1-score   support

        ADHD       0.72      0.59      0.65       108
     HEALTHY       0.54      0.68      0.60       108
         MDD       0.29      0.31      0.30       108
         OCD       0.25      0.23      0.24       1

In [8]:
# overtrained model
get_ssl_features('overtrained_acrosssub_shallownet_150feat_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet', num_extracted_features=150)

features_df.shape = (2688, 152)


,0,1,2,3,4,5,6,7,8,9,...,142,143,144,145,146,147,148,149,ID,diagnosis
0,12.635345,-3.582933,-0.185518,-7.124908,-33.094341,1.751927,-19.259064,0.980741,14.272475,3.365021,...,1.721353,7.587253,2.691187,-12.417859,-0.456325,0.006986,-3.689821,6.693016,sub-87964717,SMC
1,15.299438,-4.798579,5.185612,-6.593917,-29.050156,4.040709,-19.963778,9.726315,16.777285,1.798957,...,5.035230,6.558728,3.391831,-14.977820,0.614941,3.959617,-4.208437,3.565895,sub-87964717,SMC
2,16.772455,-3.395893,0.152670,-7.454100,-32.352108,2.944906,-22.120523,4.655383,16.944836,1.565345,...,2.464790,8.475899,2.979654,-15.630788,-0.216080,-0.985977,-3.799419,7.003053,sub-87964717,SMC


quick SVM model
[[59  5 22 21  1]
 [ 9 61 13 23  2]
 [18  2 36 33 19]
 [ 1 37 37 20 13]
 [12  4 22 12 58]]
              precision    recall  f1-score   support

        ADHD       0.60      0.55      0.57       108
     HEALTHY       0.56      0.56      0.56       108
         MDD       0.28      0.33      0.30       108
         OCD       0.18      0.19      0.18       108
         SMC       0.62      0.54      0.58       108

    accuracy                           0.43       540
   macro avg       0.45      0.43      0.44       540
weighted avg       0.45      0.43      0.44       540

0.4392455048489291

quick random forest model
[[62  6 21 18  1]
 [10 51 11 32  4]
 [21  8 25 38 16]
 [ 0 31 39 25 13]
 [ 5 10 25 11 57]]
              precision    recall  f1-score   support

        ADHD       0.63      0.57      0.60       108
     HEALTHY       0.48      0.47      0.48       108
         MDD       0.21      0.23      0.22       108
         OCD       0.20      0.23      0.22       

### pretext model 50 features

In [16]:
# best model checkpoint
get_ssl_features('acrosssub_50feat_pretext_model', dataset, df_sample, eval=True, to_disk=True, num_extracted_features=50)

features_df.shape = (2688, 52)


,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,ID,diagnosis
0,0.532304,3.829793,-0.430381,4.252481,-1.384971,-2.284594,-19.166306,-4.244578,1.721899,-2.508856,...,-1.258278,-1.058604,0.927077,0.233525,0.605244,-0.611597,-1.793311,-0.969478,sub-87964717,SMC
1,1.282263,2.542199,2.460679,4.444350,-0.421117,0.519686,-20.942211,-4.378036,1.536193,-2.673515,...,-3.298463,-1.547187,0.078922,0.734727,-0.862515,-0.289685,-2.435214,-1.241847,sub-87964717,SMC
2,1.071030,0.693853,1.960208,5.386222,0.254885,-2.035523,-23.922855,-3.678423,2.244879,-3.970068,...,-0.407650,-0.171258,0.938032,3.459286,3.073155,-0.338672,-1.132302,0.352085,sub-87964717,SMC


quick SVM model
[[57  0 13 23 15]
 [25 41  3 24 15]
 [22 35 19 24  8]
 [ 7 23 28 42  8]
 [12 28  3 16 49]]
              precision    recall  f1-score   support

        ADHD       0.46      0.53      0.49       108
     HEALTHY       0.32      0.38      0.35       108
         MDD       0.29      0.18      0.22       108
         OCD       0.33      0.39      0.35       108
         SMC       0.52      0.45      0.48       108

    accuracy                           0.39       540
   macro avg       0.38      0.39      0.38       540
weighted avg       0.38      0.39      0.38       540

0.3796044937506903

quick random forest model
[[60  3 10 19 16]
 [ 4 55  0 32 17]
 [20 30 21 18 19]
 [ 9 20 25 32 22]
 [13 24  6 23 42]]
              precision    recall  f1-score   support

        ADHD       0.57      0.56      0.56       108
     HEALTHY       0.42      0.51      0.46       108
         MDD       0.34      0.19      0.25       108
         OCD       0.26      0.30      0.28       

In [15]:
# overtrained model
get_ssl_features('overtrained_acrosssub_50feat_pretext_model', dataset, df_sample, eval=True, to_disk=True, num_extracted_features=50)

features_df.shape = (2688, 52)


,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,ID,diagnosis
0,0.985169,3.891889,-1.663651,3.693137,-1.054052,-2.293855,-16.974684,-4.104893,1.525412,-2.654570,...,-0.131403,-0.581822,-0.064482,-0.245294,0.507439,-0.253126,-0.792511,-1.782067,sub-87964717,SMC
1,1.417029,4.115095,1.167675,4.628066,-0.842520,0.439773,-19.010715,-4.591773,2.259824,-2.434310,...,-2.745334,-2.295626,0.041776,0.951865,-1.172435,1.165308,-1.550761,-1.830126,sub-87964717,SMC
2,1.300080,2.458149,-0.347941,4.680026,1.513699,-2.501022,-22.114927,-3.981549,2.064567,-3.587168,...,-0.174688,-0.942412,0.703044,2.498853,2.830074,0.088156,0.008734,-0.577674,sub-87964717,SMC


quick SVM model
[[57  0  9 27 15]
 [24 40  5 23 16]
 [20 38 22 18 10]
 [ 7 14 27 43 17]
 [14 27  2 21 44]]
              precision    recall  f1-score   support

        ADHD       0.47      0.53      0.50       108
     HEALTHY       0.34      0.37      0.35       108
         MDD       0.34      0.20      0.25       108
         OCD       0.33      0.40      0.36       108
         SMC       0.43      0.41      0.42       108

    accuracy                           0.38       540
   macro avg       0.38      0.38      0.38       540
weighted avg       0.38      0.38      0.38       540

0.37595825877971795

quick random forest model
[[60  3  8 21 16]
 [ 3 53  2 30 20]
 [19 24 33 17 15]
 [ 8 18 22 37 23]
 [18 31  4 12 43]]
              precision    recall  f1-score   support

        ADHD       0.56      0.56      0.56       108
     HEALTHY       0.41      0.49      0.45       108
         MDD       0.48      0.31      0.37       108
         OCD       0.32      0.34      0.33      

### pretext model with soft margin loss (0.25 dropout)

In [6]:
# best model checkpoint
get_ssl_features('acrosssub_soft_margin_loss_pretext_model', dataset, df_sample, eval=True, to_disk=True)

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-7.529795,-4.775673,12.061558,10.331050,0.569838,2.952614,0.439917,0.167060,5.213090,-1.864353,...,3.128101,8.402712,-2.372483,1.281823,-8.790417,-0.179843,2.697247,-4.472449,sub-87964717,SMC
1,-6.932564,-3.487274,9.676962,9.289244,0.574574,2.927734,-0.353255,-0.186739,5.203309,-2.249730,...,2.614557,5.515316,-3.588432,2.151644,-6.597898,-0.385852,2.548435,-3.753773,sub-87964717,SMC
2,-7.055388,-3.230823,9.919414,11.759319,-0.670908,3.545580,1.525463,-0.568282,5.000744,-2.417435,...,2.675336,6.487484,-3.388251,1.319001,-6.869210,0.183566,2.301235,-1.508480,sub-87964717,SMC


quick SVM model
[[59  5 13 26  5]
 [21 56 10  9 12]
 [26 16 37 14 15]
 [ 1 49 30 24  4]
 [ 8  6 63 17 14]]
              precision    recall  f1-score   support

        ADHD       0.51      0.55      0.53       108
     HEALTHY       0.42      0.52      0.47       108
         MDD       0.24      0.34      0.28       108
         OCD       0.27      0.22      0.24       108
         SMC       0.28      0.13      0.18       108

    accuracy                           0.35       540
   macro avg       0.35      0.35      0.34       540
weighted avg       0.35      0.35      0.34       540

0.3397957970483333

quick random forest model
[[62  4  2 30 10]
 [19 50 10 17 12]
 [19 14 30 19 26]
 [ 0 53 26 26  3]
 [12  4 52 14 26]]
              precision    recall  f1-score   support

        ADHD       0.55      0.57      0.56       108
     HEALTHY       0.40      0.46      0.43       108
         MDD       0.25      0.28      0.26       108
         OCD       0.25      0.24      0.24       

In [7]:
# overtrained model
get_ssl_features('overtrained_acrosssub_soft_margin_loss_pretext_model', dataset, df_sample, eval=True, to_disk=True)

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-5.855083,-7.727406,12.221320,14.037726,0.102029,2.820677,-0.149468,0.302036,3.921639,-1.482393,...,3.979167,10.372499,-2.736665,2.408197,-8.681751,0.079202,2.404514,-5.526348,sub-87964717,SMC
1,-5.438762,-5.820913,9.628739,11.997428,-0.018043,1.948442,-0.738979,-0.414632,4.591846,-1.989791,...,3.682869,6.663427,-3.572170,2.588218,-5.981269,0.278089,2.353071,-3.891693,sub-87964717,SMC
2,-5.352138,-5.791450,9.795975,15.059916,-1.232315,2.726134,0.533534,-0.766387,4.567248,-1.457844,...,4.474850,8.624177,-3.688293,1.963357,-6.804841,1.191428,1.767730,-2.902045,sub-87964717,SMC


quick SVM model
[[59  7  9 28  5]
 [23 47 12 15 11]
 [25 17 41 12 13]
 [ 1 50 29 24  4]
 [ 8  5 60 18 17]]
              precision    recall  f1-score   support

        ADHD       0.51      0.55      0.53       108
     HEALTHY       0.37      0.44      0.40       108
         MDD       0.27      0.38      0.32       108
         OCD       0.25      0.22      0.23       108
         SMC       0.34      0.16      0.22       108

    accuracy                           0.35       540
   macro avg       0.35      0.35      0.34       540
weighted avg       0.35      0.35      0.34       540

0.33888672949571375

quick random forest model
[[60  3  8 26 11]
 [21 38 14 23 12]
 [19 17 38 15 19]
 [ 2 52 31 22  1]
 [11  0 55 20 22]]
              precision    recall  f1-score   support

        ADHD       0.53      0.56      0.54       108
     HEALTHY       0.35      0.35      0.35       108
         MDD       0.26      0.35      0.30       108
         OCD       0.21      0.20      0.21      

### pretext ShallowNet soft margin loss (0.25 dropout)

In [7]:
# best model checkpoint
get_ssl_features('acrosssub_shallownet_soft_margin_loss_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-12.045370,5.203001,-11.809478,10.129691,8.730456,-21.288074,7.905180,-4.753532,-0.015335,4.449780,...,-1.717194,3.936685,-1.770861,-0.964303,0.071061,-4.244002,3.329021,-0.901170,sub-87964717,SMC
1,-7.660279,7.952641,-9.445018,8.615876,7.914999,-23.735994,9.997600,-2.478247,1.605439,3.734462,...,-4.338929,4.628870,-3.563835,3.887651,-3.129733,-4.949217,5.112733,1.884892,sub-87964717,SMC
2,-8.643456,3.235795,-9.060029,14.220202,5.835453,-22.255981,9.004423,-2.686008,5.372664,3.006578,...,-2.934388,5.794709,-2.228645,2.879202,-0.315918,-6.630397,-0.330105,-2.471468,sub-87964717,SMC


quick SVM model
[[60  5 13 25  5]
 [ 5 72 12 16  3]
 [23 14 35 17 19]
 [ 0 48 27 30  3]
 [14 14 19 14 47]]
              precision    recall  f1-score   support

        ADHD       0.59      0.56      0.57       108
     HEALTHY       0.47      0.67      0.55       108
         MDD       0.33      0.32      0.33       108
         OCD       0.29      0.28      0.29       108
         SMC       0.61      0.44      0.51       108

    accuracy                           0.45       540
   macro avg       0.46      0.45      0.45       540
weighted avg       0.46      0.45      0.45       540

0.44881558138406347

quick random forest model
[[61  5 12 23  7]
 [ 4 66 14 22  2]
 [19 13 30 26 20]
 [ 1 47 32 21  7]
 [10 15 24  8 51]]
              precision    recall  f1-score   support

        ADHD       0.64      0.56      0.60       108
     HEALTHY       0.45      0.61      0.52       108
         MDD       0.27      0.28      0.27       108
         OCD       0.21      0.19      0.20      

In [8]:
# overtrained model
get_ssl_features('overtrained_acrosssub_shallownet_soft_margin_loss_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-15.678644,9.881209,-11.283293,6.507824,6.651593,-25.578022,7.420304,-5.025426,4.770994,8.980202,...,1.512510,11.096058,2.533412,-1.348197,2.579418,-12.475011,0.659687,2.148772,sub-87964717,SMC
1,-8.375675,13.341908,-7.975436,4.789265,3.798092,-27.715631,10.972767,-1.603010,4.743442,10.318605,...,1.428555,12.228336,0.587876,2.809478,2.397911,-10.055373,4.477692,6.736988,sub-87964717,SMC
2,-11.862940,9.155365,-7.279435,12.187583,3.091933,-25.843403,9.684085,-2.054261,8.364610,10.849251,...,-1.228022,10.962969,3.048387,2.483402,5.038177,-8.818765,-5.082454,3.343171,sub-87964717,SMC


quick SVM model
[[61  8 19 20  0]
 [ 9 70 14 12  3]
 [19  7 33 26 23]
 [ 0 62 13 28  5]
 [15  8  7 12 66]]
              precision    recall  f1-score   support

        ADHD       0.59      0.56      0.58       108
     HEALTHY       0.45      0.65      0.53       108
         MDD       0.38      0.31      0.34       108
         OCD       0.29      0.26      0.27       108
         SMC       0.68      0.61      0.64       108

    accuracy                           0.48       540
   macro avg       0.48      0.48      0.47       540
weighted avg       0.48      0.48      0.47       540

0.4727488749067528

quick random forest model
[[61 13 18 14  2]
 [ 4 59 15 29  1]
 [24 18 13 27 26]
 [ 0 42 33 21 12]
 [ 3 16 13 12 64]]
              precision    recall  f1-score   support

        ADHD       0.66      0.56      0.61       108
     HEALTHY       0.40      0.55      0.46       108
         MDD       0.14      0.12      0.13       108
         OCD       0.20      0.19      0.20       

### pretext model with soft margin loss (10 epochs)

In [8]:
# best model checkpoint
get_ssl_features('acrosssub_10ep_soft_margin_loss_pretext_model', dataset, df_sample, eval=True, to_disk=True)

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-1.185726,-1.812652,5.068885,0.246980,-2.601562,1.166090,-2.460453,1.014840,4.117381,0.738197,...,-2.899080,-7.798763,-1.278432,-1.515464,-3.018791,15.429169,3.204520,0.910752,sub-87964717,SMC
1,0.033688,-0.387711,4.267061,1.124661,-1.836862,0.044589,-0.043063,0.803140,2.214555,1.553338,...,-3.567886,-7.010677,-2.683650,-0.171279,-2.612827,17.178402,2.143798,1.743032,sub-87964717,SMC
2,-0.483011,0.152234,4.092942,0.670455,-1.647230,0.816747,0.704018,0.984571,3.125106,0.942811,...,-3.160301,-7.528048,-2.740133,0.843616,-1.651058,16.694237,1.527334,2.125000,sub-87964717,SMC


quick SVM model
[[59 18  7 13 11]
 [ 1 57  6 27 17]
 [ 9 18 23 34 24]
 [ 0 51 30 15 12]
 [11  5  5  3 84]]
              precision    recall  f1-score   support

        ADHD       0.74      0.55      0.63       108
     HEALTHY       0.38      0.53      0.44       108
         MDD       0.32      0.21      0.26       108
         OCD       0.16      0.14      0.15       108
         SMC       0.57      0.78      0.66       108

    accuracy                           0.44       540
   macro avg       0.43      0.44      0.43       540
weighted avg       0.43      0.44      0.43       540

0.4268945162457028

quick random forest model
[[62  6 18 14  8]
 [ 2 45  8 38 15]
 [ 7 18 31 32 20]
 [ 0 36 38 22 12]
 [ 5  6 43 12 42]]
              precision    recall  f1-score   support

        ADHD       0.82      0.57      0.67       108
     HEALTHY       0.41      0.42      0.41       108
         MDD       0.22      0.29      0.25       108
         OCD       0.19      0.20      0.19       

In [9]:
# overtrained model
get_ssl_features('overtrained_acrosssub_10ep_soft_margin_loss_pretext_model', dataset, df_sample, eval=True, to_disk=True)

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-1.330847,-3.088368,6.051607,-0.310206,-2.076479,0.169632,-2.489274,1.178272,4.181692,0.570766,...,-2.514752,-8.293884,-0.184375,-1.302711,-2.548679,15.433952,2.838049,0.641423,sub-87964717,SMC
1,0.127345,-1.546696,4.803869,0.438345,-1.315916,-1.010393,0.150936,1.616889,2.439004,1.251327,...,-3.269745,-7.268375,-1.956525,-0.088381,-2.118301,17.572662,1.327049,1.884496,sub-87964717,SMC
2,-0.189558,-1.203649,4.795016,-0.336626,-0.929377,-0.361057,1.000550,1.881567,3.439887,0.089260,...,-2.845836,-7.916263,-1.761527,1.068087,-1.135421,17.265411,0.589541,1.947669,sub-87964717,SMC


quick SVM model
[[60 18  7 12 11]
 [ 1 59  2 29 17]
 [14 15 18 33 28]
 [ 1 58 24 11 14]
 [13  8  3  3 81]]
              precision    recall  f1-score   support

        ADHD       0.67      0.56      0.61       108
     HEALTHY       0.37      0.55      0.44       108
         MDD       0.33      0.17      0.22       108
         OCD       0.12      0.10      0.11       108
         SMC       0.54      0.75      0.63       108

    accuracy                           0.42       540
   macro avg       0.41      0.42      0.40       540
weighted avg       0.41      0.42      0.40       540

0.4025391648115971

quick random forest model
[[61 11 13 14  9]
 [ 1 41  6 42 18]
 [ 7 17 32 32 20]
 [ 2 42 32 19 13]
 [ 7  7 33 15 46]]
              precision    recall  f1-score   support

        ADHD       0.78      0.56      0.66       108
     HEALTHY       0.35      0.38      0.36       108
         MDD       0.28      0.30      0.29       108
         OCD       0.16      0.18      0.17       

### pretext model with soft margin loss (3 epochs)

In [10]:
# best model checkpoint
get_ssl_features('acrosssub_3ep_soft_margin_loss_pretext_model', dataset, df_sample, eval=True, to_disk=True)

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,22.240875,-2.774086,10.166421,-2.734844,-20.334148,4.683058,-1.842080,-0.716484,3.789842,-0.530341,...,-2.135378,1.713170,2.878568,-0.334627,-1.462004,-1.275851,1.567474,-0.333517,sub-87964717,SMC
1,19.033426,-2.217509,10.272071,-3.890897,-16.428486,5.538733,-0.886464,-1.512452,3.135611,-1.985455,...,-1.226656,1.212124,3.571429,-0.577601,-1.085243,0.270354,0.655387,-1.460791,sub-87964717,SMC
2,20.169655,-3.538530,8.025307,-2.360705,-18.272587,4.484413,-0.634366,-0.157521,3.838518,-1.168988,...,-1.228914,1.314870,2.982244,-0.001873,-1.395818,-1.726364,0.948865,-0.374595,sub-87964717,SMC


quick SVM model
[[53 11 30  4 10]
 [ 3 33  5 44 23]
 [ 9 11 28 30 30]
 [ 0 18 27 18 45]
 [12 22  3 15 56]]
              precision    recall  f1-score   support

        ADHD       0.69      0.49      0.57       108
     HEALTHY       0.35      0.31      0.33       108
         MDD       0.30      0.26      0.28       108
         OCD       0.16      0.17      0.16       108
         SMC       0.34      0.52      0.41       108

    accuracy                           0.35       540
   macro avg       0.37      0.35      0.35       540
weighted avg       0.37      0.35      0.35       540

0.3505702716765301

quick random forest model
[[59 14 26  5  4]
 [ 5 26  4 52 21]
 [ 9 23 24 34 18]
 [ 6 16 26 24 36]
 [ 8  9  8 46 37]]
              precision    recall  f1-score   support

        ADHD       0.68      0.55      0.61       108
     HEALTHY       0.30      0.24      0.27       108
         MDD       0.27      0.22      0.24       108
         OCD       0.15      0.22      0.18       

In [11]:
# overtrained model
get_ssl_features('overtrained_acrosssub_3ep_soft_margin_loss_pretext_model', dataset, df_sample, eval=True, to_disk=True)

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,22.240875,-2.774086,10.166421,-2.734844,-20.334148,4.683058,-1.842080,-0.716484,3.789842,-0.530341,...,-2.135378,1.713170,2.878568,-0.334627,-1.462004,-1.275851,1.567474,-0.333517,sub-87964717,SMC
1,19.033426,-2.217509,10.272071,-3.890897,-16.428486,5.538733,-0.886464,-1.512452,3.135611,-1.985455,...,-1.226656,1.212124,3.571429,-0.577601,-1.085243,0.270354,0.655387,-1.460791,sub-87964717,SMC
2,20.169655,-3.538530,8.025307,-2.360705,-18.272587,4.484413,-0.634366,-0.157521,3.838518,-1.168988,...,-1.228914,1.314870,2.982244,-0.001873,-1.395818,-1.726364,0.948865,-0.374595,sub-87964717,SMC


quick SVM model
[[53 11 30  4 10]
 [ 3 33  5 44 23]
 [ 9 11 28 30 30]
 [ 0 18 27 18 45]
 [12 22  3 15 56]]
              precision    recall  f1-score   support

        ADHD       0.69      0.49      0.57       108
     HEALTHY       0.35      0.31      0.33       108
         MDD       0.30      0.26      0.28       108
         OCD       0.16      0.17      0.16       108
         SMC       0.34      0.52      0.41       108

    accuracy                           0.35       540
   macro avg       0.37      0.35      0.35       540
weighted avg       0.37      0.35      0.35       540

0.3505702716765301

quick random forest model
[[60 14 28  4  2]
 [ 7 21 10 48 22]
 [ 9 22 24 28 25]
 [ 2 11 28 26 41]
 [ 8  7  8 46 39]]
              precision    recall  f1-score   support

        ADHD       0.70      0.56      0.62       108
     HEALTHY       0.28      0.19      0.23       108
         MDD       0.24      0.22      0.23       108
         OCD       0.17      0.24      0.20       

### pretext model with soft margin loss (150 features)

In [12]:
# best model checkpoint
get_ssl_features('acrosssub_150feat_soft_margin_loss_pretext_model', dataset, df_sample, eval=True, to_disk=True, num_extracted_features=150)

features_df.shape = (2688, 152)


,0,1,2,3,4,5,6,7,8,9,...,142,143,144,145,146,147,148,149,ID,diagnosis
0,-11.013163,-2.091952,-0.754555,5.430296,1.437447,-3.404385,0.154880,1.820009,1.522059,1.170023,...,-2.105203,-0.543057,-2.778214,0.245180,-2.521306,-0.064656,1.595937,2.878217,sub-87964717,SMC
1,-9.135051,-3.131505,-0.448994,3.926103,2.140814,-1.957871,-1.274686,1.618623,1.282301,1.610223,...,-1.305158,-0.875738,-2.207716,-0.567321,-2.166052,-0.568555,0.867209,1.666896,sub-87964717,SMC
2,-6.380183,-2.706604,0.422875,1.756941,1.687053,-3.623837,-1.334724,3.064804,0.841953,1.148472,...,-1.216941,-0.543375,-3.546489,-3.584837,-4.308726,-0.400823,1.415380,0.387051,sub-87964717,SMC


quick SVM model
[[62  6 10 18 12]
 [21 52  3 12 20]
 [14 27 17 14 36]
 [ 4 47 29 17 11]
 [ 4 30 52  6 16]]
              precision    recall  f1-score   support

        ADHD       0.59      0.57      0.58       108
     HEALTHY       0.32      0.48      0.39       108
         MDD       0.15      0.16      0.16       108
         OCD       0.25      0.16      0.19       108
         SMC       0.17      0.15      0.16       108

    accuracy                           0.30       540
   macro avg       0.30      0.30      0.29       540
weighted avg       0.30      0.30      0.29       540

0.29490342668337044

quick random forest model
[[69 13  7 10  9]
 [16 66  4  5 17]
 [18 18 22 18 32]
 [ 8 38 23 14 25]
 [ 3 30 55  6 14]]
              precision    recall  f1-score   support

        ADHD       0.61      0.64      0.62       108
     HEALTHY       0.40      0.61      0.48       108
         MDD       0.20      0.20      0.20       108
         OCD       0.26      0.13      0.17      

In [13]:
# overtrained model
get_ssl_features('overtrained_acrosssub_150feat_soft_margin_loss_pretext_model', dataset, df_sample, eval=True, to_disk=True, num_extracted_features=150)

features_df.shape = (2688, 152)


,0,1,2,3,4,5,6,7,8,9,...,142,143,144,145,146,147,148,149,ID,diagnosis
0,-10.468291,-1.732355,0.341479,4.108348,3.223325,-3.020484,-0.500779,2.004912,1.712551,1.110271,...,-2.859026,-0.844783,-2.657928,0.793354,-3.167115,0.016587,1.367866,3.301150,sub-87964717,SMC
1,-9.754599,-1.935057,0.716301,2.785853,4.041155,-2.372832,-1.186751,2.527348,0.768679,1.590828,...,-1.867083,-1.739313,-1.871639,-0.142649,-3.156472,-0.317873,1.252581,2.470677,sub-87964717,SMC
2,-6.853885,-2.359985,1.545487,0.807781,2.882714,-2.942299,-0.875146,3.333212,1.048014,0.913970,...,-2.409781,-1.628225,-2.899913,-2.839725,-4.747772,-0.190376,1.226963,0.614625,sub-87964717,SMC


quick SVM model
[[58 16  5 21  8]
 [18 54  4 14 18]
 [20 22 20 12 34]
 [ 4 49 27 16 12]
 [ 7 30 52  6 13]]
              precision    recall  f1-score   support

        ADHD       0.54      0.54      0.54       108
     HEALTHY       0.32      0.50      0.39       108
         MDD       0.19      0.19      0.19       108
         OCD       0.23      0.15      0.18       108
         SMC       0.15      0.12      0.13       108

    accuracy                           0.30       540
   macro avg       0.29      0.30      0.29       540
weighted avg       0.29      0.30      0.29       540

0.28546456589167535

quick random forest model
[[61 16  8 14  9]
 [19 57 13  7 12]
 [19 14 32 18 25]
 [ 8 33 28 25 14]
 [ 4 25 62  5 12]]
              precision    recall  f1-score   support

        ADHD       0.55      0.56      0.56       108
     HEALTHY       0.39      0.53      0.45       108
         MDD       0.22      0.30      0.25       108
         OCD       0.36      0.23      0.28      